In [ ]:
### PROGRAMA DE SCRAPPING DE INFORMAÇÕES DO METACRITIC VOLTADO PARA JOGOS
### DEVERA PEGAR O NOME DO JOGO, SUA NOTA MEDIA DA CRITICA E DOS USUARIOS
### DATA DE LANÇAMENTO DO JOGO. E ORGANIZA-LOS EM ORDEM DECRESCENTE.

In [ ]:
### CRIANDO POR: VICTOR KRONNER
### DATA DE CRIAÇÃO: 04/03/2022

In [ ]:
### URL: https://www.metacritic.com/browse/games/score/metascore/all/all/filtered

In [47]:
import re
import requests
import pandas as pd
import firebase_admin
#from firebase_admin import db
#from firebase_admin import credentials
from bs4 import BeautifulSoup

In [48]:
page_counter = 0

URL = "https://www.metacritic.com/browse/games/score/metascore/all/all/filtered/?page="

df = pd.DataFrame

my_data = {}

thisHeaders = {'User-Agent': 'Chrome/39.0.2171.95'}

lst_names = []
lst_plataform = []
lst_release = []
lst_meta_score = []
lst_user_score = []

plataform = []
games = []
releases = []
meta_scores = []
user_scores = []

In [49]:
"""
cred = credentials.Certificate('firebaseToken.json')

firebase_admin.initialize_app(cred,{
    'databaseURL':'https://scrappermetacritic-default-rtdb.firebaseio.com/'
})
"""

"\ncred = credentials.Certificate('firebaseToken.json')\n\nfirebase_admin.initialize_app(cred,{\n    'databaseURL':'https://scrappermetacritic-default-rtdb.firebaseio.com/'\n})\n"

In [50]:
def URL_Request(myURL, MyPage_counter):
  myFinalURL = "{0}{1}".format(URL,MyPage_counter)  
  return myFinalURL

In [51]:
def Get_Game_Names(soup):
  for table in soup.find_all("table"):
    for game in table.find_all("h3"):
      games.append(game.text)
  return games

In [52]:
def Get_Plataform_Game(soup):
  for table in soup.find_all("td"):
    for info_data in table.find_all("div", {"class": "clamp-details"}):
      for data in info_data.find_all("span", {"class": "data"}):
        aux = data.text
        aux = re.sub(r"\s+","",aux)
        plataform.append(aux)
  return plataform

In [53]:
def Get_Meta_Score(soup):

  for table in soup.find_all("table"):
    for nota_info in table.find_all("div", {"class": "browse-score-clamp"}):
      for nota_meta in nota_info.find_all("div", {"metascore_w large game positive", "metascore_w large game mixed", "metascore_w large game negative", "metascore_w large game tbd"}):
          meta_scores.append(nota_meta.text)
  return meta_scores

In [54]:
def Get_User_Score(soup):

  for table in soup.find_all("table"):
    for nota_info in table.find_all("div", {"class": "browse-score-clamp"}):
      for nota_user in nota_info.find_all("div", {"metascore_w user large game positive", "metascore_w user large game mixed", "metascore_w user large game negative", "metascore_w user large game tbd"}):
          user_scores.append(nota_user.text)
  return user_scores

In [55]:
def Get_Data_From_Game(soup):
  for table in soup.find_all("td"):
    for info_data in table.find_all("div", {"class": "clamp-details"}):
      for data in info_data.find_all("span", {"class": ""}):
        releases.append(re.split("\n", data.text))
  return releases

In [56]:
def Start_Scrapping(myURL, myHeaders):
  conteudo_html = requests.get(myURL, headers = myHeaders).text

  soup = BeautifulSoup(conteudo_html, "lxml")

  print(soup.title.text)

  global lst_names
  global lst_plataform
  global lst_meta_score
  global lst_user_score
  global lst_release

  lst_names = Get_Game_Names(soup)
  lst_plataform = Get_Plataform_Game(soup)
  lst_meta_score = Get_User_Score(soup)
  lst_user_score = Get_Meta_Score(soup)
  lst_release = Get_Data_From_Game(soup)
  return

In [57]:
"""
def CreatePandaStructure(myGameName, myGamePlataform, myMetaScore, myUserScore, myGameData):
  
  data = {
          'Name': myGameName,
          'Plataform': myGamePlataform,
          'Meta Score': myMetaScore,
          'User Score': myUserScore,
          'Release': myGameData
          }

  return data
"""

"\ndef CreatePandaStructure(myGameName, myGamePlataform, myMetaScore, myUserScore, myGameData):\n  \n  data = {\n          'Name': myGameName,\n          'Plataform': myGamePlataform,\n          'Meta Score': myMetaScore,\n          'User Score': myUserScore,\n          'Release': myGameData\n          }\n\n  return data\n"

In [58]:
"""
def PushToBD(nomes, plataforrmas, datas, notasMeta, notasUser):
    
    ref = db.reference('/Jogos/')
    for each in range(len(nomes)):
        eachGame = "game{0}".format(each)        
        ref.push({
          'Game_{0}'.format(each + 1): {
              'lancamento':'{0}'.format(datas[each]),
              'nome':'{0}'.format(nomes[each]),
              'nota_meta':'{0}'.format(notasMeta[each]),
              'nota_publico':'{0}'.format(notasUser[each]),
              'plataforma':'{0}'.format(plataforrmas[each])
          }  
        })
    return
"""

'\ndef PushToBD(nomes, plataforrmas, datas, notasMeta, notasUser):\n    \n    ref = db.reference(\'/Jogos/\')\n    for each in range(len(nomes)):\n        eachGame = "game{0}".format(each)        \n        ref.push({\n          \'Game_{0}\'.format(each + 1): {\n              \'lancamento\':\'{0}\'.format(datas[each]),\n              \'nome\':\'{0}\'.format(nomes[each]),\n              \'nota_meta\':\'{0}\'.format(notasMeta[each]),\n              \'nota_publico\':\'{0}\'.format(notasUser[each]),\n              \'plataforma\':\'{0}\'.format(plataforrmas[each])\n          }  \n        })\n    return\n'

In [81]:
def __main__():
  exit = False
  html_content = None

  global page_counter
  global my_data

  page_counter = 0

  while exit != True:

      html_content = requests.get(URL_Request(URL,page_counter), headers = thisHeaders)

      if html_content.status_code == 200:
        # Code 200 Significa Transação bem sucedida logo foi possivel pegar o conteudo HTML do Site
        html_content = requests.get(URL_Request(URL,page_counter), headers = thisHeaders)
        Start_Scrapping(URL_Request(URL,page_counter), thisHeaders)
      else:
        # Print "Negado" caso o acesso seja bloqueado pelo Site
        print("Negado")
        exit = True
        break

      if page_counter == 1:
        #Page Counter Limiter Utilizar um limite pois as paginas sem dados começam a partir da pagina 193
        #Logo um limite impede trabalho exagerado sem razão.
        exit = True
        break

      page_counter += 1

  #PushToBD(lista_nomes, lista_plataformas, lista_datas, lista_notas_meta, lista_notas_usuarios)
  
  my_data = {"Names" : lst_names, "Plataform" : lst_plataform, "Meta Score" : lst_meta_score, "User Score" : lst_user_score, "Release" : lst_release}

  df = pd.DataFrame(data = my_data)

  df

  return

#TODO: DATA FRAME NOT WORKING

  #Imprime os dados em formato de Data Frame podendo ser organizado da maneira que for mais a calhar
  #ao usuario.

In [82]:
__main__()

Best Video Games of All Time - Metacritic
Best Video Games of All Time - Page 2 - Metacritic


In [83]:
df

pandas.core.frame.DataFrame